In [1]:
#from serial import Serial
import serial
import matplotlib.pyplot as plt
import time
import numpy as np


s_sensor = serial.Serial(port = "COM7", baudrate=115200,bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)
s_printer = serial.Serial(port = "COM11", baudrate=250000)


"""Mecmesin Advanced Force Gauge Code"""
s_AFG = serial.Serial(port = "COM10", baudrate=115200,bytesize=8, timeout=2, stopbits=serial.STOPBITS_ONE)
s_AFG.write(0x2A)#* 42 0x2A Continuous transmit
def read_force():
    """Reads the force from Mecmesin AFG on s_AFG"""
    time.sleep(0.05)
    s_AFG.flushInput()
    s_AFG.write(0x3F)#? 63 0x3F Transmit the current reading
    force_N = s_AFG.readline().decode().strip()
    time.sleep(0.05)
    s_AFG.flushInput()
    s_AFG.write(0x3F)#? 63 0x3F Transmit the current reading
    force_N = s_AFG.readline().decode().strip()
    while force_N == '':#Incase empty bit arrives
        s_AFG.flushInput()
        s_AFG.write(0x3F)#? 63 0x3F Transmit the current reading
        force_N = s_AFG.readline().decode().strip()
    force_N = abs(float(force_N))#AFG reports compression as negative
    #print(force_N)
    return force_N

In [2]:
s_piezo = serial.Serial(port = "COM6", baudrate=9600)
def read_force():
    s_piezo.flushInput()
    force_IDK = s_piezo.readline().decode().strip()
    while force_IDK == "":
        force_IDK = s_piezo.readline().decode().strip()
    return int(force_IDK)

In [3]:
def read_printer():
    """Reads printers current position over serial"""
    s_printer.flushInput()
    s_printer.write("M114 R\n".encode())#M114 returns cooridnated
    grbl_out = s_printer.readline().decode()
    #print(grbl_out)
    while(grbl_out[0] != "X"):#Sometimes temperature readings get transmitted instead
        grbl_out = s_printer.readline().decode()
    #print(grbl_out)
    data = grbl_out.split()[0:3]#X, Y, Z
    data = [float(d[2:]) for d in data]
    return data

In [4]:
def read_sensor():
    """Reads ReSkin Sensor from arduino over serial"""
    s_sensor.flushInput()
    serialString = s_sensor.readline()
    serialString = serialString.decode('Ascii')
    #time.sleep(0.001)
    
    b20 = [float(b) for b in serialString.split()]
    return b20

In [5]:
def setpos(X,Y,Z):
    """Sets position of 3d printer via serial (Marlin)"""
    line = "G0 "+ " X"+str(X)+" Y"+str(Y)+ " Z"+str(Z)+  " F" + str(feedrate)+"\n"
    s_printer.write(line.encode()) # Send g-code block to grbl
    line = "M400 \n"
    s_printer.write(line.encode())#M400 halts gcode until move is completed
    line = "M118 moved!\n"
    s_printer.write(line.encode()) #M118 asks serial to send from 3d (but only after M400/move is completed)
    grbl = s_printer.readline().decode()
    while grbl != 'moved!\r\n':#Dont continue if we have not moved
        grbl = s_printer.readline().decode()
    return

In [6]:
#Setup
# Wake up grbl
s_printer.write("\r\n\r\n".encode())
time.sleep(2)   # Wait for grbl to initialize
s_printer.flushInput()  # Flush startup text in serial input

#Set established 0/0/0 pos
"""Sets origin for printer - set position to be above bottom left screw and 2mm above sensor surface (X & Y + 10)"""
feedrate = "1600"

print('Sending: ' + "G90")
s_printer.write("G90\n".encode())

#setpos(x_def, y_def, z_def)
print(read_printer())
print('Sending: ' + "G92")
s_printer.write("G92 X0 Y0 Z0\n".encode())
print(read_printer())
time.sleep(3)

Sending: G90
[-36.0, -63.0, -57.8]
Sending: G92
[0.0, 0.0, 0.0]


In [7]:
read_printer()

[0.0, 0.0, 0.0]

In [8]:
b20 = read_sensor()
print(b20)

[-0.15, -84.0, -36.06, 30.51, 55.65, -103.8, -41.62, 29.89, -678.9, 978.75, -670.82, 28.52, -133.65, 2264.7, 116.64, 28.23, -41.55, 70.05, -86.88, 30.2]


In [9]:
read_force()

0

In [22]:
"""Normalization Values collected before and after"""
norm_data = []
setpos(0,0,0)
time.sleep(1)

for i in range(5000):
    b20 = read_sensor()
    if i%1000 == 0:
        print(i)
        print("B20 Read")
        print(b20)
    norm_data.append(b20)

0
B20 Read
[4.5, -90.75, -52.03, 30.66, 49.65, -97.5, -39.93, 29.87, -637.2, 998.4, -633.07, 28.61, -146.1, 2249.7, 190.21, 28.16, -34.8, 79.95, -86.88, 30.09]
1000
B20 Read
[10.35, -88.8, -47.92, 30.64, 47.55, -94.05, -37.75, 29.76, -635.25, 996.45, -644.93, 28.45, -149.7, 2247.15, 191.91, 28.3, -36.3, 73.8, -90.75, 30.07]
2000
B20 Read
[3.45, -89.85, -33.15, 30.75, 50.1, -93.9, -45.74, 29.85, -636.9, 1003.05, -640.82, 28.38, -150.0, 2247.15, 191.91, 28.58, -34.5, 74.7, -95.59, 30.2]
3000
B20 Read
[0.45, -98.7, -48.88, 30.84, 50.7, -100.8, -45.01, 29.93, -630.9, 994.65, -646.38, 28.54, -141.45, 2244.75, 182.71, 28.36, -34.2, 76.8, -95.11, 30.15]
4000
B20 Read
[0.15, -105.15, -51.06, 30.66, 36.3, -100.35, -44.77, 29.85, -635.55, 986.55, -652.19, 28.54, -138.0, 2249.1, 192.87, 28.3, -28.65, 80.7, -90.27, 30.02]


In [23]:
print(len(norm_data))

5000


In [24]:
"""Avoid corners with screws"""
notest = []
for i in (0,1,7,8):
    for q in (0,1,7,8):
        notest.append([i,q])
notest.append([0,5,3])
print(len(notest))
print(notest)
print([0,5] in notest) 

17
[[0, 0], [0, 1], [0, 7], [0, 8], [1, 0], [1, 1], [1, 7], [1, 8], [7, 0], [7, 1], [7, 7], [7, 8], [8, 0], [8, 1], [8, 7], [8, 8], [0, 5, 3]]
False


In [25]:
truths = []
sensor_data = []

In [26]:
setpos(0,0,0)

In [27]:
iterations = 200
#print(f"Estimated time to completion: {round(170*iterations/60,0)}min")
grid_x = 9
grid_y = 9
jump_mm = 19/9
z_depths = [0.4,0.6,0.8,1,1.2,1.4]
z_depths_mm = [-z-2.4 for z in z_depths]
wait_time = 0.5
setpos(0,0,0)

no_cord = []
time_start = time.time()
time_for_iteration = 0
for iteration in range(1,iterations):
    setpos(0,0,0)
    for g_x in range(grid_x):#Iteratate over grid
        g_x_mm = round(g_x*jump_mm,2)
        setpos(g_x_mm,0,0)
        for g_y in range(grid_y):
            if not ([g_x,g_y] in notest): #Remove corners with screws
                g_y_mm = round(g_y*jump_mm,2)
                setpos(g_x_mm,g_y_mm,0)#move above testing depth
                for z_mm in z_depths_mm:
                    if not([g_x,g_y,z_mm] in notest):
                        setpos(g_x_mm,g_y_mm,z_mm)#move to testing depth

                        #Readdata
                        b20 = read_sensor()
                        sensor_data.append(b20)

                        force_N = read_force()
                        if force_N == 0:#If not touching sensor we define as 0/0/0
                            notest.append([g_x,g_y,z_mm])
                            truths.append([-1,-1,force_N])
                            print("Notest:",[g_x,g_y,z_mm,z_mm])
                        else:
                            truths.append([g_x_mm,g_y_mm,force_N])
                            print([g_x_mm,g_y_mm,force_N])
                else:
                    b20 = read_sensor()
                    sensor_data.append(b20)
                    truths.append([-1,-1,0])
                setpos(g_x_mm,g_y_mm,0)#Move above sensor again
    
    time_for_iteration = round(time.time()-time_start,0)-time_for_iteration
    if iteration == 1:
        print("Initial time: ", time_for_iteration)
        print(f"Initial time to completion: {round(time_for_iteration*(iterations-iteration)/60,0)}min")
    if iteration == 2:
        print("Improved time",time_for_iteration)
        print("Still being tested: ",notest)
        print(f"Expected time to completion: {round(time_for_iteration*(iterations-iteration)/60,0)}min")
        print(f"Expected time to completion: {round(time_for_iteration*(iterations-iteration)/60,0)}min")
        
    print(f"Iterations: {iteration}/{iterations}")
    #print(f"Current iter time: {time_for_iteration}s")
    #print([g_x_mm,g_y_mm,force_N])
    #print(b20)
setpos(0,0,0)

 


Notest: [0, 2, -2.8, -2.8]
Notest: [0, 2, -3.0, -3.0]
Notest: [0, 2, -3.2, -3.2]
[0.0, 4.22, 1]
[0.0, 4.22, 4]
[0.0, 4.22, 11]
Notest: [0, 3, -2.8, -2.8]
Notest: [0, 3, -3.0, -3.0]
[0.0, 6.33, 6]
[0.0, 6.33, 15]
[0.0, 6.33, 22]
[0.0, 6.33, 30]
[0.0, 8.44, 1]
[0.0, 8.44, 1]
[0.0, 8.44, 6]
[0.0, 8.44, 17]
[0.0, 8.44, 27]
[0.0, 8.44, 41]
Notest: [0, 5, -2.8, -2.8]
Notest: [0, 5, -3.0, -3.0]
Notest: [0, 5, -3.2, -3.2]
[0.0, 10.56, 1]
[0.0, 10.56, 9]
[0.0, 10.56, 14]
Notest: [0, 6, -2.8, -2.8]
Notest: [0, 6, -3.0, -3.0]
Notest: [0, 6, -3.2, -3.2]
Notest: [0, 6, -3.4, -3.4]
Notest: [0, 6, -3.5999999999999996, -3.5999999999999996]
Notest: [0, 6, -3.8, -3.8]
Notest: [1, 2, -2.8, -2.8]
Notest: [1, 2, -3.0, -3.0]
[2.11, 4.22, 3]
[2.11, 4.22, 7]
[2.11, 4.22, 14]
[2.11, 4.22, 19]
Notest: [1, 3, -2.8, -2.8]
Notest: [1, 3, -3.0, -3.0]
[2.11, 6.33, 17]
[2.11, 6.33, 25]
[2.11, 6.33, 34]
[2.11, 6.33, 46]
Notest: [1, 4, -2.8, -2.8]
Notest: [1, 4, -3.0, -3.0]
[2.11, 8.44, 17]
[2.11, 8.44, 22]
[2.11, 8.44

Notest: [8, 3, -2.8, -2.8]
Notest: [8, 3, -3.0, -3.0]
[16.89, 6.33, 1]
[16.89, 6.33, 8]
[16.89, 6.33, 14]
[16.89, 6.33, 24]
Notest: [8, 4, -2.8, -2.8]
Notest: [8, 4, -3.0, -3.0]
Notest: [8, 4, -3.2, -3.2]
Notest: [8, 4, -3.4, -3.4]
[16.89, 8.44, 4]
[16.89, 8.44, 11]
Notest: [8, 5, -2.8, -2.8]
Notest: [8, 5, -3.0, -3.0]
Notest: [8, 5, -3.2, -3.2]
Notest: [8, 5, -3.4, -3.4]
[16.89, 10.56, 1]
[16.89, 10.56, 1]
Notest: [8, 6, -2.8, -2.8]
Notest: [8, 6, -3.0, -3.0]
[16.89, 12.67, 1]
Notest: [8, 6, -3.4, -3.4]
Notest: [8, 6, -3.5999999999999996, -3.5999999999999996]
Notest: [8, 6, -3.8, -3.8]
Initial time:  133.0
Initial time to completion: 441.0min
Iterations: 1/200
[0.0, 4.22, 1]
[0.0, 4.22, 1]
[0.0, 4.22, 10]
Notest: [0, 3, -3.2, -3.2]
Notest: [0, 3, -3.4, -3.4]
[0.0, 6.33, 20]
[0.0, 6.33, 35]
Notest: [0, 4, -2.8, -2.8]
Notest: [0, 4, -3.0, -3.0]
Notest: [0, 4, -3.2, -3.2]
[0.0, 8.44, 6]
[0.0, 8.44, 12]
[0.0, 8.44, 25]
Notest: [0, 5, -3.4, -3.4]
Notest: [0, 5, -3.5999999999999996, -3.5999

Notest: [0, 2, -3.4, -3.4]
Notest: [0, 2, -3.5999999999999996, -3.5999999999999996]
[0.0, 4.22, 9]
Notest: [0, 3, -3.5999999999999996, -3.5999999999999996]
Notest: [0, 3, -3.8, -3.8]
Notest: [0, 4, -3.4, -3.4]
Notest: [0, 4, -3.5999999999999996, -3.5999999999999996]
[0.0, 8.44, 28]
Notest: [0, 5, -3.8, -3.8]
Notest: [1, 2, -3.5999999999999996, -3.5999999999999996]
Notest: [1, 2, -3.8, -3.8]
Notest: [1, 3, -3.5999999999999996, -3.5999999999999996]
Notest: [1, 3, -3.8, -3.8]
Notest: [1, 4, -3.2, -3.2]
Notest: [1, 4, -3.5999999999999996, -3.5999999999999996]
[2.11, 8.44, 18]
Notest: [1, 5, -3.8, -3.8]
Notest: [2, 0, -3.5999999999999996, -3.5999999999999996]
Notest: [2, 0, -3.8, -3.8]
Notest: [2, 1, -3.8, -3.8]
Notest: [2, 3, -3.4, -3.4]
[4.22, 6.33, 1]
[4.22, 6.33, 35]
Notest: [2, 4, -3.5999999999999996, -3.5999999999999996]
Notest: [2, 4, -3.8, -3.8]
Notest: [2, 5, -3.8, -3.8]
Notest: [2, 6, -3.8, -3.8]
Notest: [3, 0, -3.5999999999999996, -3.5999999999999996]
Notest: [3, 0, -3.8, -3.8]
N

KeyboardInterrupt: 

In [ ]:
print(notest)

In [38]:
setpos(3,6,-3.2)
read_force()

0.7

In [ ]:
for xyf in notest[10:15]:
    if len(xyf )> 2:
        print(xyf)
        setpos(xyf[0],xyf[1],xyf[2])
        print(read_force())
        time.sleep(5)

In [ ]:
print(f"Total time: {round(time.time()-time_start,0)}s")
print(f"Time/Iteration: {round((time.time()-time_start)/iterations,0)}s")
print(f"Sensor Data count: {len(sensor_data)}")
print(f"Truth count: {len(truths)}")

sensor_data = sensor_data[:len(truths)]

print(f"Sensor Data count: {len(sensor_data)}")
print(f"Truth count: {len(truths)}")

In [ ]:
#Normalization values:
setpos(0,0,0)
time.sleep(5)

for i in range(5000):
    b20 = read_sensor()
    if i%1000 == 0:
        print(i)
        print("B20 Read")
        print(b20)
    norm_data.append(b20)
print(len(norm_data))

In [ ]:
print(f"Sensor Data count: {len(sensor_data)}")
print(sensor_data[0:3])
print(f"Truth count: {len(truths)}")
print(truths[0:3])
print(f"Norm Value count: {len(norm_data)}")
print(norm_data[0:3])

In [ ]:
np.savetxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/norm_b20_artillery_piezo.txt", norm_data, fmt="%s")
np.savetxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/b20_artillery_piezo.txt", sensor_data, fmt="%s")
np.savetxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/truths_artillery_piezo.txt", truths, fmt="%s")

truths =  np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/truths_artillery_screw.txt",dtype = float)
sensor_data = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/b20_artillery_screw.txt",dtype = float)
norm_data = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/norm_b20_artillery_screw.txt",dtype = float) 

truths2 =  np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/truths_artillery_screw2.txt",dtype = float)
sensor_data2 = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/b20_artillery_screw2.txt",dtype = float)
norm_data2 = np.loadtxt("C:/Users/timku/Dropbox/Neural Computation Project/Models/Data/norm_b20_artillery_screw2.txt",dtype = float) 

truths = np.concatenate((truths,truths2))
sensor_data  = np.concatenate((sensor_data,sensor_data2 ))
norm_data = np.concatenate((norm_data,norm_data2))
print(len(truths))